In [1]:
import tensorflow as tf

In [2]:
#num_gpus_available = len(tf.config.experimental.list_physical_devices('GPU'))
#print("Num GPUs Available: ", num_gpus_available)
#assert num_gpus_available > 0

In [2]:
!pip install -q -U transformers

In [3]:
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification
import pandas as pd
import numpy as np

In [4]:
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
#import tensorflow_datasets as tfds
#ds = tfds.load('amazon_us_reviews/Digital_Ebook_Purchase_v1_00', split='train', shuffle_files=True)
#assert isinstance(ds, tf.data.Dataset)
#print(ds)
#ds = tfds.load('amazon_us_reviews/Digital_Ebook_Purchase_v1_00',split='train',shuffle_files=True)
#assert isinstance(ds,tf.data.Dataset)


In [5]:
#df = tfds.as_dataframe(ds)
df = pd.read_csv("/content/amazon_reviews_us_Jewelry_v1_00.tsv", sep='\t',on_bad_lines='skip')

In [6]:
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,50423057,R135Q3VZ4DQN5N,B00JWXFDMG,657335467,Everbling Purple and Clear Briolette Drop Swar...,Jewelry,5,0,0,N,Y,Beauties!,so beautiful even tho clearly not high end ......,2015-08-31
1,US,11262325,R2N0QQ6R4T7YRY,B00W5T1H9W,26030170,925 Sterling Silver Finish 6ct Simulated Diamo...,Jewelry,5,0,0,N,N,Great product.,"Great product.. I got this set for my mother, ...",2015-08-31
2,US,27541121,R3N5JE5Y4T6W5M,B00M2L6KFY,697845240,"Sterling Silver Circle ""Friends Forever"" Infin...",Jewelry,5,0,0,N,Y,Exactly as pictured and my daughter's friend l...,Exactly as pictured and my daughter's friend l...,2015-08-31
3,US,5350721,R2I150CX5IVY9Q,B0006SW2WU,569859289,Surgical Stainless Steel Domed 9mm Fishbone Ri...,Jewelry,5,0,0,N,Y,Five Stars,Love it. Fits great. Super comfortable and nea...,2015-08-31
4,US,24484424,R1RM9ICOOA9MQ3,B009YPDW70,332947422,"Sterling Silver Family Pendant Necklace, 18""",Jewelry,5,0,0,N,Y,... a Mother's Day gift for my Mom and she lov...,Got this as a Mother's Day gift for my Mom and...,2015-08-31


In [7]:
df["star_rating"].unique()

array([5, 1, 4, 3, 2])

In [8]:
df["Sentiment"] = df["star_rating"].apply(lambda score: "positive" if score >=3 else "negative")
df['Sentiment'] = df['Sentiment'].map({'positive':1, 'negative':0})
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,Sentiment
0,US,50423057,R135Q3VZ4DQN5N,B00JWXFDMG,657335467,Everbling Purple and Clear Briolette Drop Swar...,Jewelry,5,0,0,N,Y,Beauties!,so beautiful even tho clearly not high end ......,2015-08-31,1
1,US,11262325,R2N0QQ6R4T7YRY,B00W5T1H9W,26030170,925 Sterling Silver Finish 6ct Simulated Diamo...,Jewelry,5,0,0,N,N,Great product.,"Great product.. I got this set for my mother, ...",2015-08-31,1
2,US,27541121,R3N5JE5Y4T6W5M,B00M2L6KFY,697845240,"Sterling Silver Circle ""Friends Forever"" Infin...",Jewelry,5,0,0,N,Y,Exactly as pictured and my daughter's friend l...,Exactly as pictured and my daughter's friend l...,2015-08-31,1
3,US,5350721,R2I150CX5IVY9Q,B0006SW2WU,569859289,Surgical Stainless Steel Domed 9mm Fishbone Ri...,Jewelry,5,0,0,N,Y,Five Stars,Love it. Fits great. Super comfortable and nea...,2015-08-31,1
4,US,24484424,R1RM9ICOOA9MQ3,B009YPDW70,332947422,"Sterling Silver Family Pendant Necklace, 18""",Jewelry,5,0,0,N,Y,... a Mother's Day gift for my Mom and she lov...,Got this as a Mother's Day gift for my Mom and...,2015-08-31,1


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 540270 entries, 0 to 540269
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   marketplace        540270 non-null  object
 1   customer_id        540270 non-null  int64 
 2   review_id          540270 non-null  object
 3   product_id         540270 non-null  object
 4   product_parent     540270 non-null  int64 
 5   product_title      540266 non-null  object
 6   product_category   540270 non-null  object
 7   star_rating        540270 non-null  int64 
 8   helpful_votes      540270 non-null  int64 
 9   total_votes        540270 non-null  int64 
 10  vine               540270 non-null  object
 11  verified_purchase  540270 non-null  object
 12  review_headline    540264 non-null  object
 13  review_body        540036 non-null  object
 14  review_date        540268 non-null  object
 15  Sentiment          540270 non-null  int64 
dtypes: int64(6), object(

In [10]:
df["Sentiment"].unique()

array([1, 0])

In [11]:
#df['short_review'] =df['review_body'].str.decode("utf-8")
df['short_review'] =df['review_body']

In [12]:
df = df[["short_review", "Sentiment"]]
df.head()

,short_review,Sentiment
0,so beautiful even tho clearly not high end ......,1
1,"Great product.. I got this set for my mother, ...",1
2,Exactly as pictured and my daughter's friend l...,1
3,Love it. Fits great. Super comfortable and nea...,1
4,Got this as a Mother's Day gift for my Mom and...,1


In [13]:
df.shape

(540270, 2)

In [14]:
# Dropping last n rows using drop
n = 540000
df.drop(df.tail(n).index,
        inplace = True)

In [15]:
df.shape

(270, 2)

In [16]:
index = df.index
number_of_rows = len(index)
print(number_of_rows)

270


In [17]:
df.tail()

,short_review,Sentiment
265,"Pretty but very lightweight. For the price, th...",1
266,Love them! Four stars for the pearls and the w...,1
267,Very cute,1
268,These earrings are seriously amazing quality f...,1
269,This ring is so gorgeous! Ive gotten so many ...,1


In [18]:
df.head()

,short_review,Sentiment
0,so beautiful even tho clearly not high end ......,1
1,"Great product.. I got this set for my mother, ...",1
2,Exactly as pictured and my daughter's friend l...,1
3,Love it. Fits great. Super comfortable and nea...,1
4,Got this as a Mother's Day gift for my Mom and...,1


In [19]:
df = df.dropna()

In [20]:
reviews = df['short_review'].values.tolist()
labels = df['Sentiment'].tolist()

In [21]:
print(reviews[:2])
print(labels[:2])

['so beautiful even tho clearly not high end ... But super great for price and as costume jewelry, though others will have to look twice since they appear very near real ... Very nice', "Great product.. I got this set for my mother, as she is allergic to bijoux (probably to the nickel in some products, I don't know for sure, I just know that her ears get swallowed, red and itchy unless is silver or gold).<br />And I'm glad that this is sterling silver and still at a affordable price. She never got an allergic reaction, and now she wears them every day."]
[1, 1]


In [22]:
from sklearn.model_selection import train_test_split
training_sentences, validation_sentences, training_labels, validation_labels = train_test_split(reviews, labels, test_size=.2)

In [23]:
#tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
#tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
#from transformers import AutoTokenizer
#tokenizer = AutoTokenizer.from_pretrained('google/bert_uncased_L-4_H-256_A-4', model_max_length=512)

from transformers import BertModel, BertTokenizer, AdamW
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [24]:
tokenizer([training_sentences[0]], truncation=True,
                            padding=True, max_length=512)

{'input_ids': [[101, 9686, 157, 19556, 2249, 25165, 16880, 7462, 123, 151, 2137, 141, 1592, 3663, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [25]:
training_sentences[0]

'IT TURN NASTY THE 2 ND DAY'

In [26]:
train_encodings = tokenizer(training_sentences,truncation=True,padding=True)
val_encodings = tokenizer(validation_sentences,truncation=True,padding=True)

In [27]:
train_dataset = tf.data.Dataset.from_tensor_slices((
                            dict(train_encodings),
                            training_labels
                            ))
val_dataset = tf.data.Dataset.from_tensor_slices((
                            dict(val_encodings),
                            validation_labels
                            ))

In [28]:
train_dataset, val_dataset

(<_TensorSliceDataset element_spec=({'input_ids': TensorSpec(shape=(375,), dtype=tf.int32, name=None), 'token_type_ids': TensorSpec(shape=(375,), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(375,), dtype=tf.int32, name=None)}, TensorSpec(shape=(), dtype=tf.int32, name=None))>,
 <_TensorSliceDataset element_spec=({'input_ids': TensorSpec(shape=(171,), dtype=tf.int32, name=None), 'token_type_ids': TensorSpec(shape=(171,), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(171,), dtype=tf.int32, name=None)}, TensorSpec(shape=(), dtype=tf.int32, name=None))>)

In [29]:
#model = BertModel.from_pretrained('bert-base-cased')
#model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=2)
# We initialize our pre-trained BERT model
from transformers import BertTokenizer, TFBertForSequenceClassification

model = TFBertForSequenceClassification.from_pretrained('bert-base-cased',num_labels=2)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
#learning_rate = 0.00001
#optimizer =tf.keras.optimizers.Adam(learning_rate=learning_rate)
#loss= tf.keras.losses.CategoricalCrossentropy()
#acc=tf.keras.metrics.CategoricalAccuracy('accuracy')

optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-08)
model.compile(optimizer=optimizer, loss='mse', metrics=['accuracy'])

In [31]:
#optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5, epsilon=1e-08)
#model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])
#history = model.fit(train_dataset.shuffle(100).batch(16),epochs=2,batch_size=16,validation_data=val_dataset.shuffle(100).batch(16))

# We train and fine tune our pre-trained BERT model

history = model.fit(train_dataset.shuffle(100).batch(8),
          epochs=1,
          batch_size=8,
          validation_data=val_dataset.shuffle(100).batch(8), verbose=1)

Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
27/27 [==============================] - 774s 26s/step - loss: 0.2246 - accuracy: 0.5880 - val_loss: 0.1192 - val_accuracy: 0.3889


In [32]:
model.save_pretrained("./sentiment")

In [42]:
#loaded_model = TFDistilBertForSequenceClassification.from_pretrained("./sentiment/")
loaded_model = TFBertForSequenceClassification.from_pretrained("./sentiment/")

Some layers from the model checkpoint at ./sentiment/ were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at ./sentiment/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [61]:
test_sentence = "He got a job in USA and got good salary and now enjoying life"

predict_input = tokenizer.encode(test_sentence,
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf")
print(predict_input)
tf_output = loaded_model.predict(predict_input)[0]
print(tf_output)

tf_prediction = tf.nn.softmax(tf_output, axis=1)
print(tf_prediction)
labels = ['Negative','Positive']
label = tf.argmax(tf_prediction, axis=1)
label = label.numpy()
print(label)
print(labels[label[0]])

tf.Tensor(
[[  101  1124  1400   170  2261  1107  3066  1105  1400  1363 10326  1105
   1208  8965  1297   102]], shape=(1, 16), dtype=int32)
1/1 [==============================] - 0s 112ms/step
[[0.86517584 0.9423147 ]]
tf.Tensor([[0.48072487 0.5192752 ]], shape=(1, 2), dtype=float32)
[1]
Positive
